In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install super-gradients

In [ ]:
import os
import torch
import numpy as np
import cv2
from super_gradients.training import models
from super_gradients.common.object_names import Models
import matplotlib.pyplot as plt

In [ ]:
def init_HPE_model(device):
    # Initialise machine learning model
    yolo_nas_pose = models.get("yolo_nas_pose_l", pretrained_weights="coco_pose").to(device)
    return yolo_nas_pose

def predict_image(HPE_model, input_path, confidence):
    result = HPE_model.predict(input_path, conf=confidence)
    return result

def extract_poses(result):
    poses = result.prediction.poses
    landmarks = np.array([])
    for person, pose in enumerate(poses):
        landmarks = np.concatenate((landmarks, pose.flatten()))

    landmarks.flatten()
    return landmarks

def draw_keypoints(result, color):
    keypoint_colors = color * 17
    edge_colors = color * 19

    image = result.draw(
        edge_colors=edge_colors,
        joint_thickness=5,
        keypoint_colors=keypoint_colors,
        keypoint_radius=10,
        box_thickness=5,
        show_confidence=True,
    )
    return image

def annotate_image(image, text, confidence, color=(255, 0, 0)):
    font = cv2.FONT_HERSHEY_DUPLEX
    org1 = (25, 50)
    org2 = (25, 100)
    fontScale = 1
    thickness = 3
    image = cv2.putText(image, text, org1, font, fontScale,
                        color, thickness, cv2.LINE_AA, False)
    image = cv2.putText(image, str(confidence), org2, font, fontScale,
                        color, thickness, cv2.LINE_AA, False)

    return image

def save_image(filename, image):
    cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
# If GPU available, use it
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Initialize the Human Pose Estimation (HPE) model
yolo_nas_pose = init_HPE_model(device)

# Define the path to your images in Google Drive
image_folder_path = "/content/drive/My Drive/your_folder_name/"  # Replace with your folder path

# List all image files in the folder
image_files = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image
for image_path in image_files:
    # Predict keypoints on the image
    result = predict_image(yolo_nas_pose, image_path, confidence=0.3)

    # Draw keypoints on the image
    color = (0, 255, 0)  # Green color for keypoints
    image_with_keypoints = draw_keypoints(result, [color])

    # Annotate the image with confidence score
    image_with_keypoints = annotate_image(image_with_keypoints, text="Keypoints Detected", confidence=0.0, color=color)

    # Save the annotated image to Google Drive
    output_filename = os.path.join(image_folder_path, f"annotated_{os.path.basename(image_path)}")
    save_image(output_filename, image_with_keypoints)

    # Display the annotated image
    plt.imshow(cv2.cvtColor(image_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()